<a href="https://colab.research.google.com/github/Erssa001/ECGR_4105/blob/main/HW6_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

data_path = '../data-unversioned/p1ch7/'
cifar10 = cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
    ]))

label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]

100%|██████████| 170498071/170498071 [00:02<00:00, 57173507.34it/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

seq_single = nn.Sequential(
  nn.Linear(3072, 512),
  nn.Tanh(),
  nn.Linear(512, 10),
  nn.LogSoftmax(dim=1))

seq_multi = nn.Sequential(
  nn.Linear(3072, 512),
  nn.Tanh(),
  nn.Linear(512, 256),
  nn.Tanh(),
  nn.Linear(256, 128),
  nn.Tanh(),
  nn.Linear(128, 10),
  nn.LogSoftmax(dim=1))

learning_rate = 0.01
n_epochs = 300
loss_fn = nn.NLLLoss()

optimizer_single = optim.SGD(seq_single.parameters(), lr=learning_rate)
optimizer_multi = optim.SGD(seq_multi.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(1, n_epochs + 1):
  for imgs, labels in train_loader:

    batch_size = imgs.shape[0]
    outputs = seq_single(imgs.view(batch_size, -1))
    loss = loss_fn(outputs, labels)

    optimizer_single.zero_grad()
    loss.backward()
    optimizer_single.step()

  if ((epoch == 1) or (epoch % 10 == 0)):
    print(f"Epoch {epoch}, Loss {loss.item():.4f}")

Epoch 1, Loss 1.9932
Epoch 10, Loss 1.4383
Epoch 20, Loss 1.3204
Epoch 30, Loss 0.6231
Epoch 40, Loss 0.2488
Epoch 50, Loss 0.4303
Epoch 60, Loss 0.1493
Epoch 70, Loss 0.0435
Epoch 80, Loss 0.0510
Epoch 90, Loss 0.0451
Epoch 100, Loss 0.0472
Epoch 110, Loss 0.0339
Epoch 120, Loss 0.0152
Epoch 130, Loss 0.0191
Epoch 140, Loss 0.0267
Epoch 150, Loss 0.0139
Epoch 160, Loss 0.0150
Epoch 170, Loss 0.0145
Epoch 180, Loss 0.0138
Epoch 190, Loss 0.0261
Epoch 200, Loss 0.0082
Epoch 210, Loss 0.0106
Epoch 220, Loss 0.0128
Epoch 230, Loss 0.0067
Epoch 240, Loss 0.0109
Epoch 250, Loss 0.0082
Epoch 260, Loss 0.0061
Epoch 270, Loss 0.0066
Epoch 280, Loss 0.0070
Epoch 290, Loss 0.0067
Epoch 300, Loss 0.0097


In [ ]:
for epoch in range(1, n_epochs + 1):
  for imgs, labels in train_loader:

    batch_size = imgs.shape[0]
    outputs = seq_multi(imgs.view(batch_size, -1))
    loss = loss_fn(outputs, labels)

    optimizer_multi.zero_grad()
    loss.backward()
    optimizer_multi.step()

  if ((epoch == 1) or (epoch % 10 == 0)):
    print(f"Epoch {epoch}, Loss {loss.item():.4f}")

Epoch 1, Loss 1.6575
Epoch 10, Loss 1.3371
Epoch 20, Loss 1.0627
Epoch 30, Loss 0.6645
Epoch 40, Loss 0.3956
Epoch 50, Loss 0.2582
Epoch 60, Loss 0.0182
Epoch 70, Loss 0.2774
Epoch 80, Loss 0.0017
Epoch 90, Loss 0.0012
Epoch 100, Loss 0.0017
Epoch 110, Loss 0.0014
Epoch 120, Loss 0.0025
Epoch 130, Loss 0.0009
Epoch 140, Loss 0.0014
Epoch 150, Loss 0.0010
Epoch 160, Loss 0.0011
Epoch 170, Loss 0.0004
Epoch 180, Loss 0.0007
Epoch 190, Loss 0.0004
Epoch 200, Loss 0.0004
Epoch 210, Loss 0.0007
Epoch 220, Loss 0.0004
Epoch 230, Loss 0.0005
Epoch 240, Loss 0.0003
Epoch 250, Loss 0.0003
Epoch 260, Loss 0.0005
Epoch 270, Loss 0.0004
Epoch 280, Loss 0.0003
Epoch 290, Loss 0.0007
Epoch 300, Loss 0.0005


In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in train_loader:
    outputs = seq_single(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Single Layer Training Accuracy: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in val_loader:
    outputs = seq_single(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Single Layer Validation Accuracy: %f" % (correct / total))

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in train_loader:
    outputs = seq_multi(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Multi Layer Training Accuracy: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in val_loader:
    outputs = seq_multi(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Multi Layer Validation Accuracy: %f" % (correct / total))

Single Layer Training Accuracy: 1.000000
Single Layer Validation Accuracy: 0.472900
Multi Layer Training Accuracy: 1.000000
Multi Layer Validation Accuracy: 0.454400
